In [1]:
# -*- coding: utf-8 -*-
"""
# 2_Baseline_Models.ipynb

Ce notebook implémente et évalue des modèles de recommandation de base :
le filtrage collaboratif (basé sur la similarité) et la factorisation matricielle (SVD).
Ces modèles serviront de points de comparaison pour le modèle de Deep Learning.
"""

# 1. Importation des bibliothèques nécessaires
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
import matplotlib.pyplot as plt
import seaborn as sns

# Pour le filtrage collaboratif basé sur la similarité
from sklearn.metrics.pairwise import cosine_similarity

# Pour la factorisation matricielle (SVD)
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split as surprise_train_test_split
from surprise import accuracy

print("Bibliothèques importées avec succès.")

# 2. Chargement des données prétraitées et des mappings
try:
    train_df = pd.read_csv('../data/train_ratings.csv')
    val_df = pd.read_csv('../data/val_ratings.csv')
    test_df = pd.read_csv('../data/test_ratings.csv')

    user_to_id = np.load('../data/user_to_id.npy', allow_pickle=True).item()
    id_to_user = np.load('../data/id_to_user.npy', allow_pickle=True).item()
    movie_to_id = np.load('../data/movie_to_id.npy', allow_pickle=True).item()
    id_to_movie = np.load('../data/id_to_movie.npy', allow_pickle=True).item()

    print("Données d'entraînement, de validation, de test et mappings chargés avec succès.")
    print(f"Train DataFrame shape: {train_df.shape}")
    print(f"Validation DataFrame shape: {val_df.shape}")
    print(f"Test DataFrame shape: {test_df.shape}")

except FileNotFoundError:
    print("Erreur : Les fichiers de données ou de mappings n'ont pas été trouvés.")
    print("Assure-toi d'avoir exécuté le notebook '1_EDA_Preprocessing.ipynb' au préalable.")
    exit()

# Déterminer le nombre total d'utilisateurs et de films mappés
n_users_mapped = len(user_to_id)
n_movies_mapped = len(movie_to_id)
print(f"Nombre d'utilisateurs mappés : {n_users_mapped}")
print(f"Nombre de films mappés : {n_movies_mapped}")

# --- Modèle 1 : Filtrage Collaboratif Basé sur la Similarité (Simple) ---
print("\n--- Modèle 1 : Filtrage Collaboratif Basé sur la Similarité ---")

# Pour une implémentation simple, nous allons créer une matrice utilisateur-film
# à partir de l'ensemble d'entraînement.

# Créer une matrice pivot (utilisateur x film) à partir du DataFrame d'entraînement
# Remplir les valeurs manquantes avec NaN
user_movie_matrix = train_df.pivot(index='user_id_mapped', columns='movie_id_mapped', values='rating')
print("\nAperçu de la matrice utilisateur-film (sparse) :")
print(user_movie_matrix.head())

# Calcul de la similarité entre utilisateurs (basée sur les notes communes)
# Remplir les NaN avec 0 pour le calcul de similarité (ou une autre stratégie)
# Attention : Remplir avec 0 peut fausser la similarité si 0 est une note possible.
# Pour la similarité cosinus, il est souvent préférable de ne considérer que les éléments notés en commun.
# Pour simplifier ici, nous allons utiliser une approche directe sur la matrice remplie.
user_movie_matrix_filled = user_movie_matrix.fillna(0)
user_similarity = cosine_similarity(user_movie_matrix_filled)
user_similarity_df = pd.DataFrame(user_similarity, index=user_movie_matrix.index, columns=user_movie_matrix.index)
print("\nAperçu de la matrice de similarité utilisateur :")
print(user_similarity_df.head())

# Fonction de prédiction simple pour le filtrage collaboratif basé sur l'utilisateur
def predict_user_based_cf(user_id, movie_id, user_movie_matrix, user_similarity_df, k=10):
    """
    Prédit la note d'un film pour un utilisateur donné en utilisant le filtrage collaboratif basé sur l'utilisateur.
    Args:
        user_id (int): ID mappé de l'utilisateur.
        movie_id (int): ID mappé du film.
        user_movie_matrix (pd.DataFrame): Matrice utilisateur-film.
        user_similarity_df (pd.DataFrame): Matrice de similarité utilisateur.
        k (int): Nombre de voisins les plus proches à considérer.
    Returns:
        float: Prédiction de la note.
    """
    if movie_id not in user_movie_matrix.columns:
        return user_movie_matrix.mean().mean() # Retourne la moyenne générale si le film est inconnu

    # Trouver les utilisateurs similaires qui ont noté le film
    similar_users = user_similarity_df[user_id].drop(user_id).sort_values(ascending=False)

    # Filtrer les utilisateurs qui ont noté le film en question
    rated_by_similar = user_movie_matrix[movie_id].dropna()

    # Trouver les k voisins les plus proches qui ont noté le film
    # On prend les 'k' utilisateurs les plus similaires qui ont effectivement noté le film
    neighbors = similar_users.index.intersection(rated_by_similar.index)
    top_k_neighbors = similar_users[neighbors].head(k).index

    if not top_k_neighbors.empty:
        # Calcul de la moyenne pondérée des notes des voisins
        # Poids = similarité, Valeur = note du film par le voisin
        numerator = sum(user_similarity_df.loc[user, user_id] * user_movie_matrix.loc[user, movie_id]
                        for user in top_k_neighbors)
        denominator = sum(abs(user_similarity_df.loc[user, user_id]) for user in top_k_neighbors)

        if denominator == 0:
            return user_movie_matrix.loc[user_id].mean() if user_id in user_movie_matrix.index else user_movie_matrix.mean().mean()

        prediction = numerator / denominator
        # S'assurer que la prédiction est dans la plage des notes (1 à 5)
        return max(1.0, min(5.0, prediction))
    else:
        # Si aucun voisin n'a noté le film, retourner la moyenne de l'utilisateur ou la moyenne générale
        return user_movie_matrix.loc[user_id].mean() if user_id in user_movie_matrix.index else user_movie_matrix.mean().mean()

# Évaluation du modèle de filtrage collaboratif sur l'ensemble de test
print("\nÉvaluation du Filtrage Collaboratif Basé sur la Similarité (sur l'ensemble de test)...")
predictions_cf = []
true_ratings_cf = []

# Pour des raisons de performance, évaluer sur un sous-ensemble du test_df si test_df est très grand
# Ou optimiser la fonction de prédiction pour des calculs par lots.
# Ici, nous allons itérer, ce qui peut être lent pour de grands datasets.
# Limiter à 1000 prédictions pour l'exemple si le dataset est grand.
sample_test_df = test_df.sample(n=min(len(test_df), 5000), random_state=42) # Limiter à 5000 échantillons

for index, row in sample_test_df.iterrows():
    user_id = row['user_id_mapped']
    movie_id = row['movie_id_mapped']
    true_rating = row['rating']

    predicted_rating = predict_user_based_cf(user_id, movie_id, user_movie_matrix, user_similarity_df)
    predictions_cf.append(predicted_rating)
    true_ratings_cf.append(true_rating)

# Calcul des métriques
rmse_cf = sqrt(mean_squared_error(true_ratings_cf, predictions_cf))
mae_cf = mean_absolute_error(true_ratings_cf, predictions_cf)

print(f"RMSE (Filtrage Collaboratif) : {rmse_cf:.4f}")
print(f"MAE (Filtrage Collaboratif) : {mae_cf:.4f}")


# --- Modèle 2 : Factorisation Matricielle (SVD) avec Surprise ---
print("\n--- Modèle 2 : Factorisation Matricielle (SVD) avec Surprise ---")

# Charger les données dans le format attendu par Surprise
# Le Reader spécifie la plage des notes
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_ratings[['userId', 'movieId', 'rating']], reader)

# Diviser les données en entraînement et test pour Surprise
# Surprise a sa propre fonction de split qui est plus adaptée à ses objets Dataset.
# Nous allons utiliser l'ensemble d'entraînement complet pour entraîner le modèle SVD
# et l'évaluer sur l'ensemble de test que nous avons déjà préparé.
# Pour Surprise, il est plus simple de recharger les données et de faire un split interne.
# Ou alors, on peut construire un Dataset à partir de nos train_df et test_df.

# Créer un Dataset Surprise à partir de train_df
trainset = Dataset.load_from_df(train_df[['userId', 'movieId', 'rating']], reader).build_full_trainset()

# Créer un testset Surprise à partir de test_df
# Le format de testset pour Surprise est une liste de tuples (userId, movieId, true_rating)
testset = list(test_df.apply(lambda x: (x['userId'], x['movieId'], x['rating']), axis=1))

print(f"Taille du trainset Surprise : {trainset.n_ratings} ratings")
print(f"Taille du testset Surprise : {len(testset)} ratings")

# Entraînement du modèle SVD
# On peut ajuster les paramètres comme n_factors (nombre de facteurs latents), n_epochs, lr_all, reg_all
print("Entraînement du modèle SVD...")
algo_svd = SVD(n_factors=50, n_epochs=20, random_state=42, verbose=True)
algo_svd.fit(trainset)
print("Modèle SVD entraîné avec succès.")

# Évaluation du modèle SVD sur l'ensemble de test
print("\nÉvaluation du modèle SVD (sur l'ensemble de test)...")
predictions_svd = algo_svd.test(testset)

# Calcul des métriques avec Surprise
rmse_svd = accuracy.rmse(predictions_svd, verbose=False)
mae_svd = accuracy.mae(predictions_svd, verbose=False)

print(f"RMSE (SVD) : {rmse_svd:.4f}")
print(f"MAE (SVD) : {mae_svd:.4f}")

# --- Comparaison des Modèles ---
print("\n--- Comparaison des performances des modèles de base ---")
results = pd.DataFrame({
    'Modèle': ['Filtrage Collaboratif', 'SVD'],
    'RMSE': [rmse_cf, rmse_svd],
    'MAE': [mae_cf, mae_svd]
})
print(results)

# Visualisation de la comparaison
plt.figure(figsize=(10, 6))
sns.barplot(x='Modèle', y='RMSE', data=results, palette='coolwarm')
plt.title('Comparaison des RMSE des modèles de base')
plt.ylabel('RMSE')
plt.show()

plt.figure(figsize=(10, 6))
sns.barplot(x='Modèle', y='MAE', data=results, palette='coolwarm')
plt.title('Comparaison des MAE des modèles de base')
plt.ylabel('MAE')
plt.show()

# --- Sauvegarde du modèle SVD entraîné ---
from surprise import dump
dump.dump('../data/svd_model.pkl', algo=algo_svd)
print("\nModèle SVD sauvegardé sous '../data/svd_model.pkl'.")

print("\n--- Implémentation et évaluation des modèles de base terminées ! ---")

Bibliothèques importées avec succès.
Données d'entraînement, de validation, de test et mappings chargés avec succès.
Train DataFrame shape: (25600162, 6)
Validation DataFrame shape: (3200021, 6)
Test DataFrame shape: (3200021, 6)
Nombre d'utilisateurs mappés : 200948
Nombre de films mappés : 84432

--- Modèle 1 : Filtrage Collaboratif Basé sur la Similarité ---


c:\Users\abdel\OneDrive\Documents\GitHub\DeepRec\.venv\Lib\site-packages\pandas\core\reshape\reshape.py:143: RuntimeWarning: overflow encountered in scalar multiply
  num_cells = num_rows * num_columns


ValueError: negative dimensions are not allowed